In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time


### Read the file

In [3]:
df = pd.read_csv('Y2017-Y2022.csv')

C:\Users\TADIZY\AppData\Local\Temp\ipykernel_28644\1995305291.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Y2017-Y2022.csv')


In [4]:
df.head()

,FlightDate,FlightNo,ACType,Route,CgoKgs,TotalPax,BaseStation,BagPcs,ScheduledTime,FlightType,Airline,domint,Capacity,INF,CHD,ADL,ArrDep,C
0,2017-01-01,BL570,A320,SGN-BMV,0,139,BMV,21,1125,PAX,BL,True,180.0,7,6,126,A,0
1,2017-01-01,BL571,A320,BMV-SGN,0,140,BMV,11,1200,PAX,BL,True,180.0,7,15,118,D,0
2,2017-01-01,BL578,A320,BMV-VII,0,136,BMV,8,1040,PAX,BL,True,180.0,3,7,126,D,0
3,2017-01-01,BL579,A320,VII-BMV,0,132,BMV,15,1005,PAX,BL,True,180.0,6,3,123,A,0
4,2017-01-01,VJ356,A320,SGN-BMV,2898,154,BMV,22,2035,PAX,VJ,True,180.0,5,6,143,A,0


In [5]:
df = df[df['Route'].str.len() == 7]

In [6]:
set1 = set(df['Route'].str[:3])
set2 = set(df['Route'].str[4:])
Airport_list = list(set(set1) | set(set2))

In [7]:
len(Airport_list)

664

### IATA Location Search

In [8]:
lst = []
for val in Airport_list:

    url = f'https://www.iata.org/en/publications/directories/code-search/?airport.search={val}'
    r = requests.get(url)
    resp = r.content
    soup = BeautifulSoup(resp, "html.parser")
    #    print(soup)
    data = []
    try:
        for tr in soup.find('table', class_='datatable').find_all('tr')[1:]:
            row = [td.text for td in tr.find_all('td')]
            #print(row)
            lst.append(row)
    except:
        row = lst.append(['Not Available', 'Not Available', val])
df_name_search = pd.DataFrame(lst, columns=['City', 'Airport Name', 'Airport Code'])        


### Code Combine 3 Files

In [9]:
df2 = pd.DataFrame(lst, columns=['City', 'Name', 'Airport Code'])

In [10]:
df2.to_excel('IATA_Airport_Info(2017-2022).xlsx', index = False)

In [15]:
df_country_search = pd.read_excel('Airport_Location(2017-2022).xlsx')
df_lat_long_search = pd.read_excel ('AirportLat_Long(2017-2022).xlsx')

In [16]:
df_all = pd.merge(df_country_search,pd.merge(df_name_search,df_lat_long_search,how = 'inner'),how = 'inner')

In [17]:
df_all.head()

,Airport Code,Country,City,Airport Name,Lat/Long
0,IKT,Russia,Irkutsk,Irkutsk,"52.26806, 104.38889"
1,TLL,Estonia,Tallinn,Lennart Meri,"59.4133333333, 24.8322222222"
2,OKC,United States,Oklahoma City,Metropolitan Area,"Not Available, Not Available"
3,BKI,Malaysia,Kota Kinabalu,International,"5.93694, 116.05111"
4,BRU,Belgium,Brussels,Metropolitan Area,"50.9, 4.4836111111"


In [ ]:
df_all.to_excel('AirportInfo(2017-2022).xlsx')

### IATA Airline Codes Search

In [2]:
df = pd.read_csv('MissingdataAirline.csv')

In [3]:
df.head()

,Airline
0,ZH
1,ZF
2,ZE
3,ZA
4,Z5


In [36]:
# Test single-value
lst = []
val = 'VN'
url = f'https://www.iata.org/en/publications/directories/code-search/?airline.search={val}'
#print(url)
#url = f'https://flightmath.com/from-{val[:3]}-to-{val[-3:]}'
r = requests.get(url)
resp = r.content
soup = BeautifulSoup(resp, "html.parser")
#    print(soup)
data = []
try: 
    for tr in soup.find('table', class_='datatable').find_all('tr')[1:2]:
        
        row = [td.text for td in tr.find_all('td')][0:2]
        row.insert(0,val)
        lst.append(row)

except:
    row = lst.append(['Not Available', 'Not Available', val])

lst

[['VN', 'Vietnam Airlines JSC dba Vietnam Airlines JSC', 'Viet Nam']]

In [4]:
# Batch search
lst = []

for val in df['Airline'].unique():

    url = f'https://www.iata.org/en/publications/directories/code-search/?airline.search={val}'
    #print(url)
    #url = f'https://flightmath.com/from-{val[:3]}-to-{val[-3:]}'
    r = requests.get(url)
    resp = r.content
    soup = BeautifulSoup(resp, "html.parser")
    #    print(soup)
    data = []
    try:
        for tr in soup.find('table', class_='datatable').find_all('tr')[1:]:
            row = [td.text for td in tr.find_all('td')][0:2]
            row.insert(0,val)
            #print(row)
            lst.append(row)

    except:
        row = lst.append([val, 'Not Available', 'Not Available'])


In [6]:
df2 = pd.DataFrame(lst, columns=['Airline Code', 'Company Name', 'Country/Territory'])

In [7]:
df2.head()

,Airline Code,Company Name,Country/Territory
0,ZH,Shenzhen Airlines,People's Republic of China
1,ZF,AZUR air Limited Liability Company,Russian Federation
2,ZF,Safari Express Cargo Ltd,Kenya
3,ZE,Not Available,Not Available
4,ZA,Sky Angkor Airlines dba Sky Angkor Airlines Co...,Cambodia


In [ ]:
df2.to_excel('IATA_Airline_Info(2017-2022).xlsx', index = False)